In the previous notebook, we found that there is an issue with how we are doing the order routing algorithm with cvxpy. The main thing is that even though we are "forcing" eta to be either 0 or 1, the output is not what we want since an eta = 0 would mean that there should be NO swaps at all with this CFMM. In this notebook, we are going to write code that improves on this and tests this out. 

* Rewrite some of the original code in constructing the AMM, creating fees, and other stuff 
* Run through initial algorithm to get the eta values 
* Rewrite the solve function to ACTUALLY force eta to be 0 or 1 


In [42]:
import itertools
import numpy as np
import cvxpy as cp
import pandas as pd
import random

from typing import TypeVar, Tuple, Dict, List
from tqdm import tqdm   

In [43]:
TAssetId = TypeVar("TAssetId")
TNetworkId = TypeVar("TNetworkId")

class CFMM: 
    
    def __init__(self, tokens: Tuple[str, str], reserves: np.array, tx_cost: float, pool_fee: float, is_transfer: bool) -> None:
        self.tokens = tokens 
        self.reserves = reserves
        self.pool_fee = pool_fee
        self.is_transfer = is_transfer
        self.tx_cost = tx_cost
        
    def __len__(self) -> int:
        return len(self.tokens)
    
    def __str__(self) -> str: 
        return f'CFMM({self.tokens}, {self.reserves}, {self.tx_cost}, {self.pool_fee}, {self.is_transfer})'
    
    def __repr__(self) -> str:
        return str(self)

def flip(p: float) -> bool:
    return random.random() < p

def populate_chain_dict(chains: dict[TNetworkId, list[TAssetId]], center_node: TNetworkId):
    # Add tokens with denom to Center Node
    # Basic IBC transfer
    for chain, tokens in chains.items():
        if chain != center_node:
            chains[center_node].extend(f"{chain}/{token}" for token in tokens)

    1# Add tokens from Center Node to outers
    
    # Simulate IBC transfer through Composable Cosmos
    for chain, tokens in chains.items():
        if chain != center_node:
            chains[chain].extend(
                f"{center_node}/{token}"
                for token in chains[center_node]
                if f"{chain}/" not in token
            )
            
def get_mapping_matrices(all_tokens, all_cfmms): 
    count_tokens = len(all_tokens)
    
    mapping_matrices = [] 
    for cfmm in all_cfmms: 
        n_i = len(cfmm.tokens)
        A_i = np.zeros((count_tokens, n_i))
        for i, token in enumerate(cfmm.tokens):
            A_i[all_tokens.index(token), i] = 1
        mapping_matrices.append(A_i)
        
    return mapping_matrices

def solve_with_unknown_eta(origin_token: str, number_of_init_tokens: float, obj_token: str, all_tokens, all_cfmms, mapping_matrices, MAX_RESERVE=1e10):
    """In this function we solve the optimal routing problem with unknown eta values. These values are either 0 or 1. 
    How we approach this is by not including any CFMMs into the constraints that have an 
    eta value equal to 0. This way, we can solve the problem with CVXPY and not run into numerical issues. 

    Args:
        origin_token (str): The origin token
        number_of_init_tokens (float): The number of tokens we start with
        obj_token (str): The token we want to end up with
        all_tokens (_type_): A list of all tokens in the simulation
        all_cfmms (_type_): A list of all possible CFMMs in the simulation
        mapping_matrices (_type_): Matrices which map the tokens in local indices for each CFMM to global indices
    """
    # Build local-global matrices
    count_tokens = len(all_tokens)
    current_assets = np.zeros(count_tokens)  # Initial assets
    current_assets[all_tokens.index(origin_token)] = number_of_init_tokens
    
    # Getting the cfmms which have been chosen and their mapping matrices
    all_reserves = [cfmm.reserves for cfmm in all_cfmms]
    all_fees = [cfmm.pool_fee for cfmm in all_cfmms]
    all_tx_cost = [cfmm.tx_cost for cfmm in all_cfmms]
    
    deltas = [cp.Variable(len(cfmm), nonneg=True) for cfmm in all_cfmms] 
    lambdas = [cp.Variable(len(cfmm), nonneg=True) for cfmm in all_cfmms]
    
    eta = cp.Variable(len(all_cfmms), nonneg=True)
    
    psi = cp.sum([A_i @ (LAMBDA - DELTA) for A_i, DELTA, LAMBDA in zip(mapping_matrices, deltas, lambdas)])
    
    # Creating the objective 
    objective = cp.Maximize(psi[all_tokens.index(obj_token)] - cp.sum([eta[i] * all_tx_cost[i] for i in range(len(all_cfmms))]))
    
    # Creating the reserves
    new_reserves_list = [
        R + gamma * D - L for R, gamma, D, L in zip(all_reserves, all_fees, deltas, lambdas)
    ]
    
    # Creating constraints 
    constraints = [psi + current_assets >= 0]
    
    # Pool constraints
    for i, cfmm in enumerate(all_cfmms):
        new_reserves = new_reserves_list[i]
        old_reserves = all_reserves[i]
        if cfmm.is_transfer:
            constraints.append(cp.sum(new_reserves) >= cp.sum(old_reserves))
            constraints.append(new_reserves >= 0)
        else:
            # In this case we don't have a transfer pool but a regular swap
            constraints.append(cp.geo_mean(new_reserves) >= cp.geo_mean(old_reserves))
            
    for i in range(len(all_cfmms)):
        constraints.append(deltas[i] <= MAX_RESERVE * eta[i])
        
    # Set up and solve the problem
    problem = cp.Problem(objective, constraints)
    
    problem.solve(verbose=True, solver='CLARABEL', qcp=False)
    
    return deltas, lambdas, psi, eta

def solve_with_known_eta(origin_token: str, number_of_init_tokens: float, obj_token: str, all_tokens, all_cfmms, mapping_matrices, eta, MAX_RESERVE=1e10): 
    """In this function we solve the optimal routing problem with known eta values. These values are either 0 or 1. 
    How we approach this is by not including any CFMMs into the constraints that have an 
    eta value equal to 0. This way, we can solve the problem with CVXPY and not run into numerical issues. 

    Args:
        origin_token (str): The origin token
        number_of_init_tokens (float): The number of tokens we start with
        obj_token (str): The token we want to end up with
        all_tokens (_type_): A list of all tokens in the simulation
        all_cfmms (_type_): A list of all possible CFMMs in the simulation
        mapping_matrices (_type_): Matrices which map the tokens in local indices for each CFMM to global indices
        eta (_type_): An array of boolean values indicating whether a CFMM is used or not
    """
    # Build local-global matrices
    count_tokens = len(all_tokens)
    current_assets = np.zeros(count_tokens)  # Initial assets
    current_assets[all_tokens.index(origin_token)] = number_of_init_tokens
    
    # Getting the cfmms which have been chosen and their mapping matrices
    all_reserves = [cfmm.reserves for cfmm in all_cfmms]
    all_fees = [cfmm.pool_fee for cfmm in all_cfmms]
    all_tx_cost = [cfmm.tx_cost for cfmm in all_cfmms]
    
    cfmm_tx_cost = sum([eta[i] * all_tx_cost[i] for i in range(len(all_cfmms))])
    
    deltas = [cp.Variable(len(cfmm), nonneg=True) for cfmm in all_cfmms] 
    lambdas = [cp.Variable(len(cfmm), nonneg=True) for cfmm in all_cfmms]
    
    psi = cp.sum([A_i @ (LAMBDA - DELTA) for A_i, DELTA, LAMBDA in zip(mapping_matrices, deltas, lambdas)])
    
    # Creating the objective 
    objective = cp.Maximize(psi[all_tokens.index(obj_token)] - cfmm_tx_cost)
    
    # Creating the reserves
    new_reserves_list = [
        R + gamma * D - L for R, gamma, D, L in zip(all_reserves, all_fees, deltas, lambdas)
    ]
    
    # Creating the constraints
    constraints = [psi + current_assets >= 0] 
    
    # Pool constraints 
    for i, cfmm in enumerate(all_cfmms): 
        new_reserves = new_reserves_list[i]
        old_reserves = all_reserves[i]
        if cfmm.is_transfer: 
            constraints.append(cp.sum(new_reserves) >= cp.sum(old_reserves))
            constraints.append(new_reserves >= 0)
            
        else: 
            # In this case we don't have a transfer pool but a regular swap
            constraints.append(cp.geo_mean(new_reserves) >= cp.geo_mean(old_reserves))
            
    # Forcing delta and lambda to be 0 if eta is 0
    for i, cfmm in enumerate(all_cfmms): 
        if eta[i] == 0: 
            constraints.append(deltas[i] == 0)
            constraints.append(lambdas[i] == 0)
        else: 
            constraints.append(deltas[i] <= MAX_RESERVE)
    
    # Set up and solve the problem 
    problem = cp.Problem(objective, constraints)
    problem.solve(verbose=False, solver='CLARABEL', qcp=False)
    
    return deltas, lambdas, psi, objective

Setting up simulation environment

In [39]:
class OrderRoutingSimulationEnvironment: 
    
    def __init__(self, center_node: str, max_reserve: float, chains: Dict[str, List[str]]): 
        
        self.center_node = center_node
        self.max_reserve = max_reserve
        self.chains = chains
        
        # Populate the chain dictionary
        populate_chain_dict(self.chains, self.center_node)
        
        self.all_tokens = [] 
        self.all_cfmm_combinations = [] 
        self.all_cfmms = [] 
        
        for other_chain, other_tokens in chains.items():
            self.all_tokens.extend(other_tokens)
            self.all_cfmm_combinations.extend(itertools.combinations(other_tokens, 2))
            
        for cfmm in self.all_cfmm_combinations: 
            random_reserves = np.random.uniform(9500, 10051, 2)
            random_tx_cost = np.random.uniform(0, 20) 
            random_pool_fee = np.random.uniform(0.97, 0.99)
            mm = CFMM(cfmm, random_reserves, random_tx_cost, random_pool_fee, False)
            self.all_cfmms.append(mm)
            
        # Creating the CFMMs that are transfers
        for token_on_center in chains[self.center_node]:
            for other_chain, other_tokens in chains.items():
                if other_chain != self.center_node:
                    for other_token in other_tokens:
                        # Check wether the chain has the token in center, or the other way around
                        # Could cause problems if chainName == tokensName (for example OSMOSIS)
                        if other_token in token_on_center or token_on_center in other_token:
                            random_reserves = np.random.uniform(9500, 10051, 2)
                            random_tx_cost = np.random.uniform(0, 20) 
                            random_pool_fee = np.random.uniform(0.97, 0.99)
                            cfmm = CFMM((token_on_center, other_token), random_reserves, random_tx_cost, random_pool_fee, True)
                            self.all_cfmms.append(cfmm)
                            
        self.ibc_pools = [cfmm for cfmm in self.all_cfmms if cfmm.is_transfer]
        self.non_ibc_pools = [cfmm for cfmm in self.all_cfmms if not cfmm.is_transfer]
        self.num_ibc_pools = len(self.ibc_pools)
        self.num_non_ibc_pools = len(self.non_ibc_pools)
        
        self.mapping_matrices = get_mapping_matrices(self.all_tokens, self.all_cfmms)

In [12]:
chains: dict[str, list[str]] = {
    "ETHEREUM": ["WETH", "USDC", "SHIBA"],
    'CENTAURI': [],
    "OSMOSIS": ["ATOM","SCRT"],
}

sim_env = OrderRoutingSimulationEnvironment(
    center_node='CENTAURI', 
    max_reserve=1e10, 
    chains=chains
)

In [13]:
# Solving with unknnown eta values
origin_token = "WETH"
number_of_init_tokens = 2000 
obj_token = "ATOM"


deltas, lambdas, psi, eta = solve_with_unknown_eta(
    origin_token, 
    number_of_init_tokens, 
    obj_token, 
    sim_env.all_tokens, 
    sim_env.all_cfmms, 
    sim_env.mapping_matrices
)

                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) Jan 16 08:55:57 PM: Your problem has 200 variables, 91 constraints, and 0 parameters.
(CVXPY) Jan 16 08:55:57 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 16 08:55:57 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 16 08:55:57 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 16 08:55:57 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 16 08:55:57 PM: Compiling problem (target solver=CLARABEL)

/home/stephonh/.local/lib/python3.10/site-packages/cvxpy/problems/problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [68]:
t_values = sorted([eta[i].value for i in range(len(sim_env.all_cfmms))])    
results = {} 

In [69]:
for j in tqdm(range(len(t_values))):
    example_eta = [int(eta[i].value >= t_values[j]) for i in range(len(sim_env.all_cfmms))]
    
    try: 
        deltas, lambdas, psi, objective = solve_with_known_eta(
            origin_token, 
            number_of_init_tokens, 
            obj_token, 
            sim_env.all_tokens, 
            sim_env.all_cfmms, 
            sim_env.mapping_matrices,
            example_eta
        )
        
        results[j] = {
            'deltas': [delta.value for delta in deltas],
            'lambdas': [lambda_.value for lambda_ in lambdas],
            'psi': psi.value,
            'objective': objective.value, 
            'eta': example_eta,
        }
        
    except:
        print(f"Failed for t={t_values[j]}")
        continue

  0%|          | 0/40 [00:00<?, ?it/s]/home/stephonh/.local/lib/python3.10/site-packages/cvxpy/problems/problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
 92%|█████████▎| 37/40 [00:08<00:00,  4.16it/s]

Failed for t=1.6858857604427372e-07


100%|██████████| 40/40 [00:09<00:00,  4.41it/s]


Failed for t=8.274457301737546e-06


In [70]:
# Getting the best result by looking at the objective value
best_result_key = max(results, key=lambda x: results[x]['objective'])

In [71]:
best_result = results[best_result_key]

So now, we see something that is working much much better. Now, it's time to see what those swaps are and break those out. 

1. We first get the values from the best results
2. Calcualte the trades based on the psi values
3. Turn these into `Swap` objects and save to an array of `swaps`. 

In [104]:
best_eta = best_result['eta']
best_deltas = best_result['deltas']
best_lambdas = best_result['lambdas']
best_psi = best_result['psi']

chosen_cfmms = [cfmm for i, cfmm in enumerate(sim_env.all_cfmms) if best_eta[i] == 1]
chosen_mapping_matrices = [mapping_matrix for i, mapping_matrix in enumerate(sim_env.mapping_matrices) if best_eta[i] == 1]
chosen_deltas = [delta for i, delta in enumerate(best_deltas) if best_eta[i] == 1]
chosen_lambdas = [lambda_ for i, lambda_ in enumerate(best_lambdas) if best_eta[i] == 1]


In [181]:
def get_tokens_from_trade(trade, all_tokens): 
    non_zero_indices = np.nonzero(trade)[0]
    traded_tokens = [all_tokens[i] for i in non_zero_indices]
    traded_amounts = [trade[i] for i in non_zero_indices]
    
    return dict(zip(traded_tokens, traded_amounts))

In [173]:
class Swap: 
    
    def __init__(self, in_token: str, in_token_amount: float, out_token: str, out_token_amount: float, swap_type: str) -> None:
        self.in_token = in_token
        self.in_token_amount = in_token_amount
        self.out_token = out_token
        self.out_token_amount = out_token_amount
        self.swap_type = swap_type
        
    def __str__(self) -> str:
        return f'Swap({self.in_token}, {self.in_token_amount}, {self.out_token}, {self.out_token_amount}, {self.swap_type})'
    
    def __repr__(self) -> str:
        return str(self)

In [186]:
swaps = [] 

for i in range(len(chosen_cfmms)): 
    c = chosen_cfmms[i]
    m = chosen_mapping_matrices[i]
    d = chosen_deltas[i]
    l = chosen_lambdas[i]
    
    trade = m @ (l - d)
    
    tokens_from_trade = get_tokens_from_trade(trade, sim_env.all_tokens)
    
    if tokens_from_trade[c.tokens[0]] > 0:
        in_token = c.tokens[0]
        out_token = c.tokens[1]
    else: 
        in_token = c.tokens[1]
        out_token = c.tokens[0]
    
    if c.is_transfer: 
        swap_type = 'transfer'
    else: 
        swap_type = 'swap'
        
    swaps.append(Swap(in_token, tokens_from_trade[in_token], out_token, tokens_from_trade[out_token], swap_type))

In [187]:
swaps 

[Swap(OSMOSIS/ATOM, 714.3825709308805, ETHEREUM/WETH, -784.7442158189558, swap),
 Swap(OSMOSIS/SCRT, 268.7003823805993, ETHEREUM/WETH, -287.4439873754726, swap),
 Swap(ETHEREUM/SHIBA, 2.3480709210836745e-06, OSMOSIS/ATOM, -3.167116503168812e-05, swap),
 Swap(OSMOSIS/ATOM, 256.0179788798671, OSMOSIS/SCRT, -268.70033952933113, swap),
 Swap(ATOM, 789.1957860991917, CENTAURI/ETHEREUM/WETH, -865.4676713958321, swap),
 Swap(CENTAURI/ETHEREUM/USDC, 2.3838753202096597e-06, ATOM, -3.138672865043578e-05, swap),
 Swap(CENTAURI/ETHEREUM/SHIBA, 4.02110291397161e-06, CENTAURI/ETHEREUM/WETH, -4.1268990333327834e-05, swap),
 Swap(ETHEREUM/WETH, 1946.779384548574, WETH, -1999.9999956391823, transfer),
 Swap(CENTAURI/ETHEREUM/WETH, 865.467716549563, ETHEREUM/WETH, -874.5911773687386, transfer),
 Swap(ATOM, 956.0453679629642, OSMOSIS/ATOM, -970.4005157176533, transfer),
 Swap(CENTAURI/OSMOSIS/SCRT, 3.0816786228751725e-06, OSMOSIS/SCRT, -3.981850540137977e-05, transfer)]

We can see here that these swaps make more sense and we have eliminated entirely pools that are unnecessary. One thing to note is that we see that there are swaps sometimes that are very very small. We can probably put some tolerance in the objective function to have these swaps be a minimum amount since we would have to pay gas fees for something like this. 